In [ ]:
!pip install --no-deps /kaggle/input/map-utilities/bitsandbytes-0.47.0-py3-none-manylinux_2_24_x86_64.whl

In [ ]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

ROOT_PATH = os.getcwd()
if "/kaggle" in ROOT_PATH:
    ROOT_PATH = "/kaggle/input"
    sys.path.append(os.path.join(ROOT_PATH, "map-utilities"))

In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

import torch
from datasets import Dataset
from peft import PeftModel
from transformers import BitsAndBytesConfig

from utils import (
    stringify_input,
    get_model_name,
    get_sequence_classifier,
    get_tokenizer,
    get_training_arguments,
    get_trainer,
)

In [ ]:
le = joblib.load(os.path.join(ROOT_PATH, "map-utilities", "label_encoder.joblib"))
n_classes = len(le.classes_)

In [ ]:
# BASE_MODEL = "modernbert-large/transformers/default/10"
# BASE_MODEL = "qwen3-1.7b/transformers/default/1"
# BASE_MODEL = "ettin-encoder-1b/transformers/default/2"
BASE_MODEL = "qwen-3/transformers/14b/1"

MODEL_NAME = get_model_name("/kaggle" in ROOT_PATH, ROOT_PATH, BASE_MODEL)
ADAPTER_PATH = get_model_name("/kaggle" in ROOT_PATH, ROOT_PATH, "")
USE_LORA = True
USE_QLORA = True
MAX_LEN = 256

TRAIN_PATH = os.path.join(ROOT_PATH, "map-charting-student-math-misunderstandings", "train.csv")
TEST_PATH = os.path.join(ROOT_PATH, "map-charting-student-math-misunderstandings", "test.csv")

In [ ]:
qlora_config = {}
if USE_QLORA:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

In [ ]:
seq_model = get_sequence_classifier(MODEL_NAME, n_classes, qlora_config)
tokenizer = get_tokenizer(MODEL_NAME)

if "gemma" in MODEL_NAME.lower() or "qwen" in MODEL_NAME.lower():
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
    seq_model.config.pad_token_id = tokenizer.pad_token_id

if USE_LORA:
    seq_model = PeftModel.from_pretrained(seq_model, MODEL_NAME)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["stringified_input"], truncation=True)

In [ ]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

In [ ]:
print("Training Shape:", train_df.shape)
print("Testing Shape:", test_df.shape)

In [ ]:
idx = train_df.Category.str.contains("True", case=False)
tmp = train_df.loc[idx].copy()
tmp["c"] = tmp.groupby(["QuestionId", "MC_Answer"]).MC_Answer.transform("count")
tmp = tmp.sort_values("c", ascending=False)
tmp = tmp.drop_duplicates(["QuestionId"])
tmp = tmp[["QuestionId", "MC_Answer"]]
tmp["is_mc_answer_correct"] = True

train_df = train_df.merge(tmp, on=["QuestionId", "MC_Answer"], how="left")
train_df.is_mc_answer_correct = train_df.is_mc_answer_correct.fillna(False)

test_df = test_df.merge(tmp, on=["QuestionId", "MC_Answer"], how="left")
test_df.is_mc_answer_correct = test_df.is_mc_answer_correct.fillna(False)

In [ ]:
test_df["stringified_input"] = test_df.apply(
    lambda row: stringify_input(row, MODEL_NAME), axis=1
)
test_df.head()

In [ ]:
test_ds = Dataset.from_pandas(test_df[["stringified_input"]])
test_ds = test_ds.map(tokenize_function, batched=True)

In [ ]:
lengths = [len(tokenizer.encode(t, truncation=False)) for t in test_ds["stringified_input"]]

plt.hist(lengths, bins=50)
plt.title("Token Length Distribution")
plt.xlabel("Number of tokens")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

In [ ]:
L = (np.array(lengths) > MAX_LEN).sum()
print(f"There are {L} train sample(s) with more than {MAX_LEN} tokens")
np.sort(lengths)[::-1]

In [ ]:
training_args = get_training_arguments(bf16_support="/kaggle" not in ROOT_PATH)
trainer = get_trainer(
    seq_model,
    tokenizer,
    training_args,
    test_ds,
    test_ds,
)

In [ ]:
predictions = trainer.predict(test_ds)
probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=1).numpy()

In [ ]:
top3 = np.argsort(-probs, axis=1)[:, :3]

# Decode numeric class indices to original string labels
flat_top3 = top3.flatten()
decoded_labels = le.inverse_transform(flat_top3)
top3_labels = decoded_labels.reshape(top3.shape)

# Join 3 labels per row with space
joined_preds = [" ".join(row) for row in top3_labels]

# Save submission
sub = pd.DataFrame({
    "row_id": test_df.row_id.values,
    "Category:Misconception": joined_preds
})
sub.to_csv("submission.csv", index=False)
sub.head()